In [5]:
import requests
import json
import random
import resource
from pprint import pprint
from itertools import groupby
from collections import defaultdict
import sys
import pdb

The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the 11th row looks liks : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646

In [6]:
req = requests.get('https://d3c33hcgiwev3.cloudfront.net/_410e934e6553ac56409b2cb7096a44aa_SCC.txt?Expires=1484870400&Signature=LZIBHXDwvR5-66pnmEPXvHIxdHjrE5bJzw3u6CT6KfMp5sFEjmwNeiTg0Tdf4oczsZLsgdFadpcmCe2NAbJU-urGYGM2uFergLkXQOSyveQEy9oEmn8Es3nEBPPklrb4rE9YQVpaxlxv9VjLcAJXWT2HpxPGyljZqDzrJKXEGF4_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A')

In [7]:
edges = [n.split(' ')[:-1] for n in req.text.splitlines()]
graph = {}
reverse_graph = {}
for edge in edges:
    tail = int(edge[0])
    head = int(edge[1])
    if True: #tail != head:
        if graph.get(tail):
            graph[tail].append(head)
        else:
            graph[tail] = [head]
        if reverse_graph.get(head):
            reverse_graph[head].append(tail)
        else:
            reverse_graph[head] = [tail]
            
# assume that nodes are consecutively numbered.
nodes = list(range(1, len(graph)+1))
nodes.reverse()

In [ ]:
import sys
import threading

def readDirectedGraph(filename):
    f = open(filename)
    
    adjlist = []
    adjlist_reversed = []
    
    line = f.readline()
    while line != '':
        num1, num2 = line.split()
        v_from = int(num1)
        v_to = int(num2)
        max_v = max(v_from, v_to)
        
        while len(adjlist) < max_v:
            adjlist.append([])
        while len(adjlist_reversed) < max_v:
            adjlist_reversed.append([])
            
        adjlist[v_from-1].append(v_to-1)
        adjlist_reversed[v_to-1].append(v_from-1)
        
        line = f.readline()
            
    return adjlist, adjlist_reversed


t = 0
s = None
explored = None
leader = None
scc_size = 0
sorted_by_finish_time = None

def DFS_Loop_1(graph_rev, n):
    
    global t, explored, sorted_by_finish_time
    
    t = 0
    explored = [False]*n
    sorted_by_finish_time = [None]*n
    
    for i in reversed(range(n)):
        if not explored[i]:
            DFS_1(graph_rev, i)
                        
            
def DFS_1(graph_rev, i):
    
    global t, explored
    
    explored[i] = True
    
    for v in graph_rev[i]:
        if not explored[v]:
            DFS_1(graph_rev, v)
    
    sorted_by_finish_time[t] = i
    t += 1
    
    
def DFS_Loop_2(graph):
    
    global scc_size, explored, sorted_by_finish_time
    
    explored = [False]*len(graph)
    res = []
    
    for i in reversed(range(len(graph))):
        if not explored[sorted_by_finish_time[i]]:
            scc_size = 0
            DFS_2(graph, sorted_by_finish_time[i])
            res.append(scc_size)
            
    return res
    
    
def DFS_2(graph, i):
    
    global explored, scc_size
    
    explored[i] = True
    
    for v in graph[i]:
        if not explored[v]:
            DFS_2(graph, v)
    
    scc_size += 1
    

def kosarajuSCCsizes(graph, graph_rev):
    
    DFS_Loop_1(graph_rev, len(graph))
    res = DFS_Loop_2(graph)
    
    return res


def main():
    print('start')
    graph, graph_rev = readDirectedGraph('SCC.txt')
    print('start 1')
    res = kosarajuSCCsizes(graph, graph_rev)
    print('start 2')
    print(','.join(map(lambda x: str(x), sorted(res)[::-1][:5])))


if __name__ == '__main__':
    threading.stack_size(67108864) 
    sys.setrecursionlimit(2 ** 20)
    thread = threading.Thread(target = main)
    thread.start() 

In [82]:
def traverse_iter(graph, current, visited, junctions):
    # finished traversing
    if len(finishing_times) == vertexes:
        return
    else:
        print("in {}".format(current))
        group1 = [x for x in get_edges_from(graph, current) if x not in visited]
        group2 = [x for x in get_edges_to(graph, current) if x not in visited]
        possibilities = group1 + group2
        visited.add(current)
        
        # junction
        if len(possibilities) > 1:
            junctions.append(current)
            if group1 != []:
                next_node = group1[0]
            else:
                print("adding1 {}".format(current))
                finishing_times.append(current)
                next_node = group2[0]
            traverse_iter(graph, next_node, visited, junctions)
            
        # 1-way
        elif len(possibilities) == 1:
            if group2 != []:
                if current not in finishing_times:
                    print("adding2 {}".format(current))
                    finishing_times.append(current)
            junctions.append(current)
            next_node = possibilities[0]
            traverse_iter(graph, next_node, visited, junctions)
            
        # backtracking, add to finishing times
        else:
            print("adding3 {}".format(current))
            finishing_times.append(current)
            if junctions != []:
                next_node = junctions.pop()
                traverse_iter(graph, next_node, visited, junctions)
            else:
                return

def traverse_iter2(graph, current, visited):
    #print("in {}".format(current))
    if current in visited:
        return
    visited.add(current)
    possibilities = get_edges_to(graph, current) + get_edges_from(graph, current)
    for i in possibilities:
        traverse_iter2(graph, i, visited)

def traverse_iter3(graph, current, visited):
    to_visit = []
    possibilities = get_edges_to(graph, current) + get_edges_from(graph, current)
    to_visit.append(current)
    
    while len(to_visit) > 0:
        current = to_visit.pop()
        if current in visited:
            continue
        visited.add(current)
        
        possibilities = get_edges_to(graph, current) + get_edges_from(graph, current)
        for next_node in possibilities:
            to_visit.append(next_node)

In [101]:
reverse_test = {
    1:[7],
    2:[5],
    3:[9],
    4:[1],
    5:[8],
    6:[3, 8],
    7:[4, 9],
    8:[2],
    9:[6],
}
test = {
    1:[4],
    2:[8],
    3:[6],
    4:[7],
    5:[2],
    6:[9],
    7:[1],
    8:[5, 6],
    9:[3, 7],
}

# assume that nodes are consecutively numbered.
nodes = list(range(1, len(test)+1))
nodes.reverse()

class Track(object):
    """Keeps track of the current time, current source, component leader,
    finish time of each node and the explored nodes."""
 
    def __init__(self):
        self.current_time = 0
        self.current_source = None
        self.leader = {}
        self.finish_time = {}
        self.explored = set()

def dfs_outer(graph, nodes, track):
    for node in nodes:
        if node not in track.explored:
            track.current_source = node
            dfs_inner2(graph, node, track)
            
def dfs_inner(graph, node, track):
    track.explored.add(node)
    #print("explored: {}".format(node))
    track.leader[node] = track.current_source
    if graph.get(node):
        for head in graph[node]:
            if head not in track.explored:
                dfs_inner(graph, head, track)
    print("finished: {}".format(node))
    track.current_time += 1
    track.finish_time[node] = track.current_time
    
def dfs_inner2(graph, node, track):
    #print("explored: {}".format(node))
    track.leader[node] = track.current_source
    to_visit = [node]
    while len(to_visit) > 0:
        current = to_visit.pop()
        if current in track.explored:
            track.current_time += 1            
            track.finish_time[current] = track.current_time
            continue
        track.explored.add(current)
        track.leader[current] = track.current_source
        if graph.get(current):
            next_nodes = [current] + [x for x in graph[current] if x not in track.explored]
            to_visit.extend(next_nodes)
        else:
            track.current_time += 1            
            track.finish_time[current] = track.current_time
                
def scc(graph, reverse_graph, nodes):
    out = defaultdict(list)
    track = Track()
    dfs_outer(reverse_graph, nodes, track)
    sorted_nodes = sorted(track.finish_time, key=track.finish_time.get, reverse=True)
    track.current_time = 0
    track.current_source = None
    track.explored = set()
    dfs_outer(graph, sorted_nodes, track)
    for leader, vertex in groupby(sorted(track.leader, key=track.leader.get), key=track.leader.get):
        out[leader] = list(vertex)
    return out

nodes = list(range(1, len(graph)+1))
nodes.reverse()
#rtn = scc(test, reverse_test, nodes)
rtn = scc(graph, reverse_graph, nodes)
print("finished")

finished


In [102]:
rtn2 = sorted(rtn, key=lambda k: len(rtn[k]), reverse=True)

In [107]:
':'.join([str(len(rtn[key])) for key in rtn2[:5]])

'444273:970:459:314:211'